<a href="https://colab.research.google.com/github/rajakumar05032000/Book-recommendation-system/blob/main/Book_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Input, Embedding,  Dense, Concatenate,SimpleRNN,GRU,Flatten,LSTM,Dropout,Bidirectional,GlobalMaxPool1D,LeakyReLU
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!cp -r '/content/drive/MyDrive/Preprocessed_data.csv.zip' '/content'
#!unzip Preprocessed_data.csv.zip


In [ ]:
data1=pd.read_csv("Preprocessed_data.csv")

In [ ]:
data1.head()

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [ ]:
data1.drop(['location','img_s','img_m','img_l','publisher','city','state','country','Unnamed: 0'],inplace=True,axis=1)

In [ ]:
h=data1[data1.user_id==8]
h

,user_id,age,isbn,rating,book_title,book_author,year_of_publication,Summary,Language,Category
1,8,34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,"In a small town in Canada, Clara Callan reluct...",en,['Actresses']
15,8,34.7439,0060973129,0,Decision in Normandy,Carlo D'Este,1991.0,"Here, for the first time in paperback, is an o...",en,['1940-1949']
18,8,34.7439,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,"Describes the great flu epidemic of 1918, an o...",en,['Medical']
29,8,34.7439,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999.0,A look at the incredibly well-preserved ancien...,en,['Design']
30,8,34.7439,0399135782,0,The Kitchen God's Wife,Amy Tan,1991.0,A Chinese immigrant who is convinced she is dy...,en,['Fiction']
63,8,34.7439,0425176428,0,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,"Essays by respected military historians, inclu...",en,['History']
68,8,34.7439,0671870432,0,PLEADING GUILTY,Scott Turow,1993.0,9,9,9
71,8,34.7439,0679425608,0,Under the Black Flag: The Romance and the Real...,David Cordingly,1996.0,"An authoritative, revisionist history of the g...",en,['Fiction']
72,8,34.7439,074322678X,5,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,"Now back in print, Ann Beattie&#39;s finest sh...",en,['Fiction']
73,8,34.7439,0771074670,0,Nights Below Station Street,David Adams Richards,1988.0,Another story based in the fictional rural tow...,en,['Fiction']


In [ ]:
data1.columns

Index(['user_id', 'age', 'isbn', 'rating', 'book_title', 'book_author',
       'year_of_publication', 'Summary', 'Language', 'Category'],
      dtype='object')

In [ ]:
data1.isnull().sum()

user_id                0
age                    0
isbn                   0
rating                 0
book_title             0
book_author            0
year_of_publication    0
Summary                0
Language               0
Category               0
dtype: int64

In [ ]:
users = len(data1.user_id.unique())
print(users)

92107


In [ ]:
print(data1.shape)

(1031175, 10)


In [ ]:
books=len(data1.isbn.unique())
print(books)

270170


In [ ]:
YearOfPublication_to_index={j:i+1 for i,j in enumerate(data1['year_of_publication'].unique())}
index_to_YearOfPublication={i+1:j for i,j in enumerate(data1['year_of_publication'].unique())}

In [ ]:
Book_to_index={j:i+1 for i,j in enumerate(data1['isbn'].unique())}
index_to_Book={i+1:j for i,j in enumerate(data1['isbn'].unique())}

In [ ]:
Category_to_index={j:i+1 for i,j in enumerate(data1['Category'].unique())}
index_to_Category={i+1:j for i,j in enumerate(data1['Category'].unique())}

In [ ]:
Language_to_index={j:i+1 for i,j in enumerate(data1['Language'].unique())}
index_to_Language={i+1:j for i,j in enumerate(data1['Language'].unique())}

In [ ]:
User_to_index={j:i+1 for i,j in enumerate(data1['user_id'].unique())}
index_to_User={i+1:j for i,j in enumerate(data1['user_id'].unique())}

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(max_features=250)
# summary_vector=vectorizer.fit_transform(data1.Summary).toarray()
# summary_vector.shape

In [ ]:
data1=data1[data1['rating'] != 0]
print(data1.shape)

(383852, 10)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
texts=data1["Summary"].values
max_num_words = 5000
max_seq_length = 300
tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [ ]:
text_data = pad_sequences(sequences, maxlen=max_seq_length)
#!wget https://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove.6B.zip
import os
embedding_index = {}
glove_dir = '/content/'
f = open(os.path.join(glove_dir,'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))


found word vecs:  400000


In [ ]:
embedding_dim = 50
embedding_matrix = np.zeros((len(word_index)+1,embedding_dim))
embedding_matrix.shape

for word,i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
data1['Yearofpublication_index']=[YearOfPublication_to_index.get(i) for i in data1['year_of_publication']]
data1['Book_index']=[Book_to_index.get(i) for i in data1['isbn']]
data1['Category_index']=[Category_to_index.get(i) for i in data1['Category']]
data1['Language_index']=[Language_to_index.get(i) for i in data1['Language']]
data1['User_index']=[User_to_index.get(i) for i in data1['user_id']]
data1.drop(['user_id','isbn','Category','Language','year_of_publication','book_author','book_title','Summary'],inplace=True,axis=1)

In [ ]:
data1.head()

,age,rating,Yearofpublication_index,Book_index,Category_index,Language_index,User_index
1,34.7439,5,2,2,2,1,2
3,34.7439,8,2,2,2,1,4
5,30.0000,8,2,2,2,1,6
8,34.7439,9,2,2,2,1,9
9,34.7439,9,2,2,2,1,10


In [ ]:
from sklearn.preprocessing import  MinMaxScaler
scaler=MinMaxScaler()
scaler2=MinMaxScaler()


In [ ]:
data1=data1.head(1031175)

In [ ]:
text_data = pad_sequences(sequences, maxlen=max_seq_length)

In [ ]:
print(text_data.shape)
print(data1.shape)

(383852, 300)
(383852, 7)


In [ ]:
data1.head()

,age,rating,Yearofpublication_index,Book_index,Category_index,Language_index,User_index
1,34.7439,5,2,2,2,1,2
3,34.7439,8,2,2,2,1,4
5,30.0000,8,2,2,2,1,6
8,34.7439,9,2,2,2,1,9
9,34.7439,9,2,2,2,1,10


In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_summary,test_summary = train_test_split(data1,text_data, test_size=0.2)

In [ ]:
other_train=train_data.drop(['User_index','Book_index','rating'],axis=1)
other_train[['age','Yearofpublication_index','Category_index','Language_index']]=scaler.fit_transform(other_train[['age','Yearofpublication_index','Category_index','Language_index']])
other_train=other_train.values
train_summary.shape

(307081, 300)

In [ ]:
other_train.shape

(307081, 4)

In [ ]:
from keras.layers import Concatenate
from tensorflow import keras
from tensorflow.keras import regularizers
book1 = Input(shape=[1])
book_emb= Embedding(books+1, 5)(book1)
book_flat= Flatten()(book_emb)

user = Input(shape=[1])
user_emb = Embedding(users+1, 5)(user)
user_flat= Flatten()(user_emb)

conc1=Concatenate()([book_flat,user_flat])


text = Input(shape=(max_seq_length,))
text_emb = Embedding(len(word_index)+1,embedding_dim,weights=[embedding_matrix],input_length=max_seq_length,trainable=True)(text)
l1 =LSTM(16,return_sequences=True)(text_emb)
l2 = GlobalMaxPool1D()(l1)

other_proc = Input(shape=other_train[1].shape,name='op')
other1 = Dense(32,kernel_regularizer='l2')(other_proc)
lr1=LeakyReLU()(other1)
other2= Dropout(0.7)(lr1)
conc2= Concatenate()([conc1,lr1])


l3=  Dense(32,kernel_regularizer='l2')(l2)
lr2=LeakyReLU()(l3)
conc3= Concatenate()([conc2,lr2])
c3=Dropout(0.7)(conc3)
l4 = Dense(8)(c3)
l5= LeakyReLU()(l4)



out = Dense(1)(l5)
model1 = Model([book1, user,text,other_proc], out)
optimizer = keras.optimizers.Adam()
model1.compile(loss='mse', optimizer=optimizer)

In [ ]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 50)      4338650     input_3[0][0]                    
______________________________________________________________________________________________

In [ ]:
m=model1.fit([train_data.Book_index, train_data.User_index,train_summary,other_train], train_data.rating, epochs=5, batch_size=120,validation_split=0.1,verbose=1)

Epoch 1/5
2304/2304 [==============================] - 139s 59ms/step - loss: 5.3844 - val_loss: 2.9641
Epoch 2/5
2304/2304 [==============================] - 124s 54ms/step - loss: 2.9585 - val_loss: 2.7679
Epoch 3/5
2304/2304 [==============================] - 125s 54ms/step - loss: 2.7115 - val_loss: 2.7028
Epoch 4/5
2304/2304 [==============================] - 127s 55ms/step - loss: 2.5549 - val_loss: 2.6709
Epoch 5/5
2304/2304 [==============================] - 124s 54ms/step - loss: 2.4401 - val_loss: 2.6817


In [ ]:
other_test=test_data.drop(['User_index','Book_index','rating'],axis=1)
other_test[['age','Yearofpublication_index','Category_index','Language_index']]=scaler.fit_transform(other_test[['age','Yearofpublication_index','Category_index','Language_index']])
other_test=other_test.values
model_predictions = model1.predict([test_data.Book_index,test_data.User_index,test_summary,other_test])

In [ ]:
for i in range(0,50):
  print("Model_rating:",model_predictions[i]," Original Rating:",test_data.rating.iloc[i])

Model_rating: [7.939085]  Original Rating: 7
Model_rating: [7.276123]  Original Rating: 4
Model_rating: [7.0649133]  Original Rating: 6
Model_rating: [8.626167]  Original Rating: 10
Model_rating: [7.377743]  Original Rating: 10
Model_rating: [7.541521]  Original Rating: 8
Model_rating: [7.4792795]  Original Rating: 7
Model_rating: [8.002579]  Original Rating: 9
Model_rating: [7.15345]  Original Rating: 6
Model_rating: [9.478658]  Original Rating: 10
Model_rating: [7.4199634]  Original Rating: 4
Model_rating: [8.729465]  Original Rating: 10
Model_rating: [7.9504046]  Original Rating: 7
Model_rating: [6.722909]  Original Rating: 8
Model_rating: [7.417386]  Original Rating: 8
Model_rating: [7.4233]  Original Rating: 9
Model_rating: [6.6304493]  Original Rating: 7
Model_rating: [7.3374496]  Original Rating: 7
Model_rating: [7.9675922]  Original Rating: 9
Model_rating: [5.916942]  Original Rating: 8
Model_rating: [7.956137]  Original Rating: 5
Model_rating: [7.5813103]  Original Rating: 8
M

In [ ]:
model1.evaluate([test_data.Book_index,test_data.User_index,test_summary,other_test], test_data.rating)

2400/2400 [==============================] - 46s 19ms/step - loss: 2.6959


2.6959023475646973

In [ ]:
import sklearn.metrics as metrics
mae = metrics.mean_absolute_error(test_data.rating, model_predictions)
mse = metrics.mean_squared_error(test_data.rating, model_predictions)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(test_data.rating,model_predictions)

print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

MAE: 1.292996139816801
MSE: 2.6931800287224856
RMSE: 1.641091109208287
R-Squared: 0.19779854659579865


In [ ]:
#MAE- -0.93%
#MSE- 2.39%
#RMSE- 1.2%
#R-Squared- 16.56%